# Training de Modelo 1.1. – Clasificador de CV con BERT

Qué es BERT?

BERT (Bidirectional Encoder Representations from Transformers) es un modelo **transformer preentrenado** en grandes cantidades de texto.
“Bidirectional” significa que lee el texto **tanto hacia adelante como hacia atrás**, captando mejor el contexto.
Se puede **fine-tunear** para tareas específicas, como clasificación de textos, sin necesidad de entrenarlo desde cero.

**Ventajas para CVs:**

* Entiende mejor frases y contexto que TF-IDF.
* Puede captar relaciones entre frases y palabras clave (por ejemplo, “Python” + “Data Science”).
* Mejora la generalización y evita tanto el overfitting como la pérdida de semántica.


In [9]:
import requests
print(requests.get("https://huggingface.co").status_code)


200


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from transformers import (
    TFAutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding
)

Importación del df

In [11]:
import pandas as pd

df = pd.read_csv('/Users/alexmiliandieguez/code/TechCareer/data/model1_cv_role/3.processed/v1_english/cv_labeled_final.csv')
df = df.loc[:, ['cv_text', 'role_label_final']]
df.rename(columns={'cv_text': 'cv', 'role_label_final': 'role'}, inplace=True)


Codificar etiquetas

In [12]:
le = LabelEncoder()
df['role_encoded'] = le.fit_transform(df['role'])
num_classes = len(le.classes_)


Train test

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    df['cv'].values,
    df['role_encoded'].values,
    test_size=0.2,
    random_state=42,
    stratify=df['role_encoded']
)


Modelo y tokenizer

In [15]:
model_name = "google/bert_uncased_L-6_H-512_A-8"

tokenizer = AutoTokenizer.from_pretrained(model_name)

Modelo BERT

In [17]:
num_classes = len(le.classes_)

model = TFAutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_classes,
    from_pt=True  # <-- aquí indiquem que carregui pesos PyTorch
)

ValueError: Due to a serious vulnerability issue in `torch.load`, even with `weights_only=True`, we now require users to upgrade torch to at least v2.6 in order to use the function. This version restriction does not apply when loading files with safetensors.
See the vulnerability report here https://nvd.nist.gov/vuln/detail/CVE-2025-32434